In [1]:
1+1

2

In [2]:
import langchain

# Introduction to Data Ingestion

In [4]:
import os
from typing import List, Dict, Any
import pandas as pd

In [9]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter
print("Setup is done!")

/Users/neeladnatarajan/DSProjects/LLMOps/hw/RAGUdemy/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup is done!


## Understand document structure

In [11]:
## Create a simple document

doc = Document(page_content="This is a main text content that gets embedded and searched.",
               metadata={
                   "source": "Example.txt", 
                   "author": "Neeladevi", 
                   "page":1, 
                   "date": "2025-12-22"
                })

print("Document structure!")
print(f"Content: {doc.page_content}")
print(f"Metadata: {doc.metadata}")

Document structure!
Content: This is a main text content that gets embedded and searched.
Metadata: {'source': 'Example.txt', 'author': 'Neeladevi', 'page': 1, 'date': '2025-12-22'}


### Text Files(.txt) - The Simplest Case(#2-text-files)

In [12]:
## Create a simple txt file
import os
os.makedirs("data/text_files", exist_ok=True)

In [14]:
sample_texts = {
    "data/text_files/python_intro.txt": 
    
    """
    Python is a high-level, easy-to-learn programming language known for its simple and readable syntax. 
    It is widely used in web development, data science, artificial intelligence, automation, and software development. 
    Python supports multiple programming styles (procedural, object-oriented) and has a large collection of libraries, making it powerful and beginner-friendly.
    """,

    "data/text_files/machine_learning_basics.txt":

    """
    
    Machine Learning is a branch of Artificial Intelligence (AI) that enables computers to learn from data and improve automatically without being explicitly programmed. 
    It is used in applications like recommendation systems, image recognition, speech recognition, and spam detection. 
    Machine learning works by training models on data to make predictions or decisions.
    """
}

for filepath, content in sample_texts.items():
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content.strip())

print("✅ Sample text file created!")

✅ Sample text file created!


### TextLoader - Read Single File

In [18]:

from langchain_community.document_loaders import TextLoader

# Loading a single text file    
loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents= loader.load()
print(f"📄 Loaded {len(documents)} documents.")
print(f"Content: {documents[0].page_content[:300]}...")
print(f"Metadata: {documents[0].metadata}")


📄 Loaded 1 documents.
Content: Python is a high-level, easy-to-learn programming language known for its simple and readable syntax. 
    It is widely used in web development, data science, artificial intelligence, automation, and software development. 
    Python supports multiple programming styles (procedural, object-oriented) ...
Metadata: {'source': 'data/text_files/python_intro.txt'}


### DirectoryLoader for reading multiple files from a folder

In [21]:
from langchain_community.document_loaders import DirectoryLoader

## Load all the files from a directory

dir_load = DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # Pattern to match the files
    loader_cls=TextLoader, # Loader class of the files
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True
)
documents = dir_load.load()
print(f"📄 Loaded {len(documents)} documents.")
for i, doc in enumerate(documents):
    print(f"\n ------ Document {i+1} ------")
    print(f"Source: {doc.metadata['source']}")
    print(f"Length: {len(doc.page_content)} characters")

# 📊 Analysis
print("\n 📊 DirectoryLoader Characteristics:")
print("✅ Advantages:")
print(" - Loads multiple documents from a directory in one go.")
print(" - Supports glob patterns for flexible file selection.")
print(" - Progress tracking for large directories.")
print(" - Recursive loading of files in subdirectories.")
print("❌ Disadvantages:")
print(" - All files must be of the same type (e.g., all text files).")
print(" - Limited error handling per file.")
print(" - Can be memory-intensive for large directories.")



100%|██████████| 2/2 [00:00<00:00, 399.44it/s]

📄 Loaded 2 documents.

 ------ Document 1 ------
Source: data/text_files/python_intro.txt
Length: 382 characters

 ------ Document 2 ------
Source: data/text_files/machine_learning_basics.txt
Length: 374 characters

 📊 DirectoryLoader Characteristics:
✅ Advantages:
 - Loads multiple documents from a directory in one go.
 - Supports glob patterns for flexible file selection.
 - Progress tracking for large directories.
 - Recursive loading of files in subdirectories.
❌ Disadvantages:
 - All files must be of the same type (e.g., all text files).
 - Limited error handling per file.
 - Can be memory-intensive for large directories.


In [22]:
### Different Text splitting strategies

from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python is a high-level, easy-to-learn programming language known for its simple and readable syntax. \n    It is widely used in web development, data science, artificial intelligence, automation, and software development. \n    Python supports multiple programming styles (procedural, object-oriented) and has a large collection of libraries, making it powerful and beginner-friendly.'), Document(metadata={'source': 'data/text_files/machine_learning_basics.txt'}, page_content='Machine Learning is a branch of Artificial Intelligence (AI) that enables computers to learn from data and improve automatically without being explicitly programmed. \n    It is used in applications like recommendation systems, image recognition, speech recognition, and spam detection. \n    Machine learning works by training models on data to make predictions or decisions.')]


In [ ]:
### Method 1: CharacterTextSplitter

text=documents[0].page_content
text 


'Python is a high-level, easy-to-learn programming language known for its simple and readable syntax. \n    It is widely used in web development, data science, artificial intelligence, automation, and software development. \n    Python supports multiple programming styles (procedural, object-oriented) and has a large collection of libraries, making it powerful and beginner-friendly.'

In [25]:
print("\n ----- CharacterTextSplitter -----\n")
char_splitter = CharacterTextSplitter (
    separator="\n", # Split on new lines
    chunk_size=100, # Chunk size of 100 characters
    chunk_overlap=20, # Overlap between chunks 20 characters
    length_function=len, # How to measure chunk size
)

char_chunks = char_splitter.split_text(text)
print(f"Total Chunks: {len(char_chunks)}")
print(f"First Chunk: {char_chunks[0][:100]}...")

Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100



 ----- CharacterTextSplitter -----

Total Chunks: 3
First Chunk: Python is a high-level, easy-to-learn programming language known for its simple and readable syntax....


In [27]:
for i, chunk in enumerate(char_chunks):
    print(f"\n---- Chunk {i+1} ----\n")
    print(chunk)


---- Chunk 1 ----

Python is a high-level, easy-to-learn programming language known for its simple and readable syntax.

---- Chunk 2 ----

It is widely used in web development, data science, artificial intelligence, automation, and software development.

---- Chunk 3 ----

Python supports multiple programming styles (procedural, object-oriented) and has a large collection of libraries, making it powerful and beginner-friendly.


In [31]:
# Method 2: RecursiveCharacterTextSplitter
print("\n ----- RecursiveCharacterTextSplitter -----\n")
recursive_splitter = RecursiveCharacterTextSplitter (
    separators=["\n\n", "\n", " ", ""],
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

recursive_chunks = recursive_splitter.split_text(text)
print(f"Total Chunks: {len(recursive_chunks)}")
print(f"First Chunk: {recursive_chunks[0][:100]}...")

for i, chunk in enumerate(recursive_chunks):
    print(f"\n --- Chunk {i+1} --- \n")
    print(chunk)



 ----- RecursiveCharacterTextSplitter -----

Total Chunks: 6
First Chunk: Python is a high-level, easy-to-learn programming language known for its simple and readable syntax....

 --- Chunk 1 --- 

Python is a high-level, easy-to-learn programming language known for its simple and readable syntax.

 --- Chunk 2 --- 

readable syntax.

 --- Chunk 3 --- 

It is widely used in web development, data science, artificial intelligence, automation, and

 --- Chunk 4 --- 

automation, and software development.

 --- Chunk 5 --- 

Python supports multiple programming styles (procedural, object-oriented) and has a large

 --- Chunk 6 --- 

and has a large collection of libraries, making it powerful and beginner-friendly.


In [34]:
# Method 3: TokenTextSplitter
print("\n ----- TokenTextSplitter -----\n")
token_splitter = TokenTextSplitter (
    chunk_size=50,
    chunk_overlap=10,
)

token_chunks = token_splitter.split_text(text)
print(f"Total Chunks: {len(token_chunks)}")
print(f"First Chunk: {token_chunks[0][:100]}...")

for i, chunk in enumerate(token_chunks):
    print(f"\n--- Chunk {i+1} ---\n")
    print(chunk)


 ----- TokenTextSplitter -----

Total Chunks: 2
First Chunk: Python is a high-level, easy-to-learn programming language known for its simple and readable syntax....

--- Chunk 1 ---

Python is a high-level, easy-to-learn programming language known for its simple and readable syntax. 
    It is widely used in web development, data science, artificial intelligence, automation, and software development. 
 

--- Chunk 2 ---

, automation, and software development. 
    Python supports multiple programming styles (procedural, object-oriented) and has a large collection of libraries, making it powerful and beginner-friendly.


In [35]:
# 📊 Comparisons 
print("\n 📊 Text Splitting Strategies Comparison:")
print("\n CharacterTextSplitter:")
print(" ✅ Simple and predictable splitting based on character count.")
print(" ✅ Good for structured text.")
print(" ❌ May break midsentence.")
print(" 💡 Use when text has clear delimiters.")

print("\n RecursiveCharacterTextSplitter:")
print(" ✅ Respects text structure.")
print(" ✅ Tries multiple separators.")
print(" ✅ Best general-purpose splitter.")
print(" ❌ Slightly more complex.")
print(" 💡 Use for varied text formats.")

print("\n TokenTextSplitter:")
print(" ✅ Respects model token limits.")
print(" ✅ More accurate for embeddings.")
print(" ❌ Slower for character-based models.")
print(" 💡 Use when working with token-limited models.")


 📊 Text Splitting Strategies Comparison:

 CharacterTextSplitter:
 ✅ Simple and predictable splitting based on character count.
 ✅ Good for structured text.
 ❌ May break midsentence.
 💡 Use when text has clear delimiters.

 RecursiveCharacterTextSplitter:
 ✅ Respects text structure.
 ✅ Tries multiple separators.
 ✅ Best general-purpose splitter.
 ❌ Slightly more complex.
 💡 Use for varied text formats.

 TokenTextSplitter:
 ✅ Respects model token limits.
 ✅ More accurate for embeddings.
 ❌ Slower for character-based models.
 💡 Use when working with token-limited models.
